In [1]:
import pandas as pd
import pdb
import numpy as np
import geopandas as gpd
import shapely as sp
import censusgeocode as cg
from shapely import geometry

import matplotlib.pyplot as plt


In [2]:
#read tree census data from website
tree2015 = pd.read_csv('data\\2015_Street_Tree_Census.csv')
tree2005 =  pd.read_csv('data\\2005_Street_Tree_Census.csv')
tree95 =  pd.read_csv('data\\1995_Street_Tree_Census.csv')
#read 2020 census
#tracts_raw = pd.read_csv('data\\nhgis0001_csv\\nhgis0001_ds248_2020_tract.csv', encoding='latin-1')

# options are to only import columns I will need and define dtypes for them
# or keep doing it this way
def encode_boroct(bc, l, sct):
    bct = np.where(l == 1, bc+"000"+sct+"00",
                  np.where(l == 2, bc+"00"+sct +"00",
                  np.where(l == 3, bc+"0"+sct+"00",
                  np.where(l == 4, bc+sct+"00",
                  np.where(l == 5, bc+"0"+sct,
                  np.where(l == 6, bc+sct,
                                    "NaN"))))))
    return bct

    

C:\Users\Jerry\AppData\Local\Temp\ipykernel_19944\3173956785.py:3: DtypeWarning: Columns (37) have mixed types. Specify dtype option on import or set low_memory=False.
  tree2005 =  pd.read_csv('data\\2005_Street_Tree_Census.csv')


In [3]:
tree05 = tree2005.copy()
tree95 = tree95.copy()
tree15 = tree2015.copy()

ct_present = tree15["census tract"].count() + tree95["CensusTract_2010"].count() + tree15["census tract"].count() 
totals = tree95.Borough.count() + tree15.borocode.count() + tree05.borocode.count()
ct_present / totals 


0.970592516293961

In [10]:
tree05 = tree2005.copy()
tree95 = tree95[["Borough","CensusTract_2010","CensusBlock_2010","NTA_2010","SegmentID"]]
tree05 =  tree05[["cb_num","borocode",'boroname','cncldist','nta','nta_name','boro_ct','latitude','longitude','census tract', 'bin', 'bbl']]

In [11]:
missing95 = tree95[tree95.CensusTract_2010.isna()].Borough.count() #number of trees with no census tract
missing95/tree95.Borough.count()

0.04506672288965529

In [12]:
missing05 = tree05.borocode.count()-tree05.boro_ct.count()
missing05/tree05.borocode.count()

0.17777680241469887

In [116]:
tractcount = pd.DataFrame(tree2015.groupby('boro_ct').size())
#tree05 = tree2005.copy()

In [30]:
tree05 = tree05[['census tract', 'borocode','boro_ct']]
tree05 = tree05.loc[tree05['census tract'].notnull()]
tree05['sct'] = ["%.0f" % i for i in tree05['census tract'] ]
tree05['len'] = [len(i) for i in tree05['sct'] ]
tree05.borocode = [str(d) for d in tree05.borocode]
tree05['boro_ct'] = encode_boroct(tree05.borocode, tree05.len, tree05.sct)


dtype('O')

In [96]:
final05 = tree2005.copy()
tree05.count()

census tract    569458
borocode        569458
boro_ct         569458
sct             569458
len             569458
dtype: int64

In [73]:
final05.count()

OBJECTID        592372
cen_year        592372
tree_dbh        592372
address         592372
tree_loc        592372
pit_type        592372
soil_lvl        592372
status          592372
spc_latin       592372
spc_common      592372
vert_other      592372
vert_pgrd       592372
vert_tgrd       592372
vert_wall       592372
horz_blck       592372
horz_grate      592372
horz_plant      592372
horz_other      592372
sidw_crack      592372
sidw_raise      592372
wire_htap       592372
wire_prime      592372
wire_2nd        592372
wire_other      592372
inf_canopy      592372
inf_guard       592372
inf_wires       592372
inf_paving      592372
inf_outlet      592372
inf_shoes       592372
inf_lights      592372
inf_other       592372
trunk_dmg       592372
zipcode         592372
zip_city        583461
cb_num          592372
borocode        592372
boroname        592372
cncldist        592372
st_assem        592372
st_senate       592372
nta             592372
nta_name        592372
boro_ct    

In [98]:
tree95 = tree95.loc[tree95.CensusTract_2010.notnull()]
tree95['borocode'] = np.where(tree95.Borough == 'Staten Island', '5',
                     np.where(tree95.Borough == 'Queens', '4',
                     np.where(tree95.Borough == 'Brooklyn', '3',
                     np.where(tree95.Borough == 'Bronx', '2',
                     np.where(tree95.Borough == 'Manhattan', '1',
                              6)))))

                              

In [99]:
tree95.borocode.unique()

array(['1', '2', '3', '4', '5'], dtype=object)

In [100]:
tree95['sct'] = ["%.0f" % i for i in tree95['CensusTract_2010'] ]
tree95['len'] = [len(i) for i in tree95['sct'] ]
tree95.len.unique()

array([2, 3, 4, 5, 1, 6], dtype=int64)

In [101]:
tree95['boro_ct'] = encode_boroct(tree95.borocode, tree95.len, tree95.sct)
tree95['boro_ct'] = pd.to_numeric(tree95.boro_ct)
tree95.head()



,Borough,CensusTract_2010,CensusBlock_2010,NTA_2010,SegmentID,borocode,sct,len,boro_ct
0,Manhattan,48.0,2000.0,MN21,33134,1,48,2,1004800
1,Manhattan,39.0,2001.0,MN24,31567,1,39,2,1003900
2,Manhattan,39.0,2001.0,MN24,31567,1,39,2,1003900
3,Manhattan,39.0,2001.0,MN24,31567,1,39,2,1003900
4,Manhattan,39.0,2001.0,MN24,31567,1,39,2,1003900


In [119]:

tree05.boro_ct = [int(z) for z in pd.to_numeric(tree05.boro_ct)]
final95 = pd.DataFrame(tree95.groupby('boro_ct').size())
final05 = pd.DataFrame(tree05.groupby('boro_ct').size())
final95.rename(columns={0: 'trees95'}, inplace = True)
final05.rename(columns={0: 'trees05'}, inplace = True)
merged = pd.merge(final95,final05, how='inner', left_index = True, right_index = True)
len(merged)

2119

In [121]:
merged = pd.merge(merged, tractcount, how = 'inner', left_index = True, right_index= True)
merged.head()

,trees95,trees05,0_x,0_y
boro_ct,,,,
1000600,153,89,187,187
1000700,81,133,144,144
1000800,96,74,288,288
1000900,111,59,83,83
1001200,36,39,111,111


In [18]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output# Load Data
df = px.data.tips()# Build App 
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("JupyterDash Demo"),
    dcc.Graph(id='graph'),
    html.Label([
        "colorscale",
        dcc.Dropdown(
            id='colorscale-dropdown', clearable=False,
            value='plasma', options=[
                {'label': c, 'value': c}
                for c in px.colors.named_colorscales()
            ])
    ]),
])# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input("colorscale-dropdown", "value")]
)
def update_figure(colorscale):
    return px.scatter(
        df, x="total_bill", y="tip", color="size",
        color_continuous_scale=colorscale,
        render_mode="webgl", title="Tips"
    )# Run app and display result inline in the notebook
app.run_server(mode='inline')